In [1]:
## Load data
import glob
import pandas as pd
import pickle

group = 'ageAtEnroll' #race

df = pd.read_csv(r'/home/hbt/jchr_data/jchr_racial_diff/results/preprocessed_data/1_2_model_input_ws60min_ph60min_v6.csv')




# Define the path to your folder
folder_path = rf'/home/hbt/jchr_data/jchr_racial_diff/results/processed_data/2_1_1_predicted_results_rnn_v6_{group}_v2'

# Pattern to match the files
file_pattern = f'{folder_path}/patient*_ratio*.pkl'

# Dictionary to store the data
dictionary = {}

# Iterate over files matched by glob
for file_path in glob.glob(file_pattern):
    with open(file_path, 'rb') as file:
        # Extracting N and X from the filename
        filename = file_path.split('/')[-1]  # Adjust if necessary based on your OS
        patient_id, ratio_id = filename.replace('.pkl', '').split('_')
        # Extracting numeric parts from patient_id and ratio_id
        patient_num = ''.join(filter(str.isdigit, patient_id))
        ratio_num = ''.join(filter(str.isdigit, ratio_id))
        
        # Load the content of the file
        file_data = pickle.load(file)

        # Store the data
        dictionary.update(file_data)

# Now loaded_data contains all your files' data

In [2]:
import my_utils
import numpy as np
iteration = 1
dict_results = {}

for (PtID, percentage), value in dictionary.items():

    # print('Iteration:',iteration,'PtID:',PtID,'ratio:' ,percentage)
    iteration=iteration+1
    
    # My actual/true values and my baseline value
    y_actual = value['y_actual']
    y_last_value = value['y_last_val']
    y_pred_single = value['y_pred_single']
    y_pred_base = value['y_pred']
    y_pred_tl = value['y_pred_tl']
    y_last_value2 = y_last_value.values.reshape(-1, 1) # make y_last_val to nparray (672,1) from Series (672,)
        
        
    # calculating my values for each prediction base_model
    rmse_naive = my_utils.calculate_results(y_actual, y_last_value2)
    rmse_single = my_utils.calculate_results(y_actual, y_pred_single)
    rmse_base = my_utils.calculate_results(y_actual, y_pred_base)
    rmse_tl = my_utils.calculate_results(y_actual,y_pred_tl)
    
     
    dict_results[(PtID, percentage)] = {
        "ptid_test": value['PtId_test'],
        "ptid_group": value['ptid_group'],
        "rmse_naive": rmse_naive,
        "rmse_single": rmse_single,
        "rmse_base": rmse_base,
        "rmse_tl": rmse_tl
    
    }


In [3]:
import pandas as pd
df = pd.DataFrame.from_dict(dict_results, orient='index').reset_index()
df = df.rename(columns={'level_0': 'PtID', 'level_1': 'ratio'})
df

,PtID,ratio,ptid_test,ptid_group,rmse_naive,rmse_single,rmse_base,rmse_tl
0,7,90,7,female,2.011643,2.201581,1.537105,1.531627
1,83,50,83,female,2.526862,2.244087,1.986418,1.970185
2,5,10,5,female,1.277876,1.333196,1.214352,1.094050
3,120,50,120,male,1.653102,1.558303,1.446260,1.388604
4,21,80,21,female,2.721661,2.782691,2.394147,2.326223
...,...,...,...,...,...,...,...,...
2250,21,60,21,female,2.721661,2.787249,2.416983,2.345664
2251,44,50,44,male,2.035758,1.862145,1.761998,1.700406
2252,104,20,104,female,2.648725,2.753988,2.540208,2.370006
2253,8,70,8,female,4.042003,4.630995,3.283120,3.208880


In [4]:
# save dataframe
file_path_save = f'../results/processed_data/3_1_calculated_ressults/3_1_calculated_results_v6_{group}_v2.csv'
df.to_csv(file_path_save)
